## 2.2 テキストをトークン化する

In [1]:
with open("./src/the-verdict.txt") as f:
    raw_text = f.read()
print(f"文字数 : {len(raw_text)}")
print(raw_text[:99])

文字数 : 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re

# 特殊記号や空白と単語を分離
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:30])

# リストから空白を削除
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--']
4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## 2.3 トークンをトークン ID に変換する

In [3]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [4]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed =[
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!\'\)])', r'\1', text)
        text = re.sub(r'([\'\(])+\s', r'\1', text)
        return text

In [5]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It's the last he painted, you know, " Mrs. Gisburn said with pardonable pride.


# 2.4 特別なコンテキストトークンを追加する

In [6]:
text = "Hello, do you like tea?"
try:
    print(tokenizer.encode(text))
except Exception as e:
    print(f"{type(e).__name__}: {e}")

KeyError: 'Hello'


上のような未知の用語に対応する必要がある．

In [7]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endooftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


In [8]:
for item in list(vocab.items())[-5:]:
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endooftext|>', 1130)
('<|unk|>', 1131)


In [10]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed =[
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [11]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <endooftext> ".join((text1, text2))
print(text)

Hello, do you like tea? <endooftext> In the sunlit terraces of the palace.


In [12]:
tokenizer = SimpleTokenizerV2(vocab)
cipher = tokenizer.encode(text)
print(cipher)

[1131, 5, 355, 1126, 628, 975, 10, 1131, 55, 988, 956, 984, 722, 988, 1131, 7]


In [13]:
print(tokenizer.decode(cipher))

<|unk|>, do you like tea? <|unk|> In the sunlit terraces of the <|unk|>.


# 2.5 バイトペアエンコーディング

BPE (byte pair encoding): GPT-2, 3 で使われているトークン化スキーム

BPEのアルゴリズム詳細
BPEは以下のステップで動作します：

1. 初期化
すべての単語を文字単位に分割します。例えば、単語「lower」は「l」「o」「w」「e」「r」に分割されます。
2. 頻度計算
単語リストの中で出現する文字ペア（隣接する2文字）の頻度を計算します。
例えば、単語「lower」ではペア「l」と「o」、「o」と「w」、「w」と「e」、「e」と「r」の出現頻度を数えます。
3. 置換
最も頻度の高いペアを1つの新しいトークンに置換します。
例えば、「l」と「o」を「lo」に置換します。
4. 繰り返し
ステップ2と3を指定回数繰り返し、頻度の高いペアをどんどん結合していきます。
この過程で、より長いトークンが徐々に形成され、最終的に効率的なトークン分割が完了します。

In [14]:
from importlib.metadata import version
import tiktoken

print('tiktoken version:', version("tiktoken"))

tiktoken version: 0.9.0


In [15]:
tokenizer = tiktoken.get_encoding("gpt2")

In [16]:
text = (
    "Hello, do you like tea> <|endoftext|> In the sunlit terraces "
    "of someunknownPlace"
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 29, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271]


In [17]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea> <|endoftext|> In the sunlit terraces of someunknownPlace


単語を分解してトークン化することで、未知の単語 someunknownPlace を正確に decode できる．

**練習**

In [18]:
text = "Akwirw ier"
integers = tokenizer.encode(text)
print(integers)

for i in integers:
    print(tokenizer.decode([i]))

[33901, 86, 343, 86, 220, 959]
Ak
w
ir
w
 
ier


# 2.6 スライディングウインドウによるデータサンプリング

In [19]:
with open("./src/the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_sample = enc_text[50:]

context_size = 4 # 入力に含まれるトークンの数を決定する
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [21]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f"{context} ----> {desired}")

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [22]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f"{tokenizer.decode(context)} ----> {tokenizer.decode([desired])}")

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt) # テキスト全体のトークン化
        
        # The Verdict を max_length の長さに分割する
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self): # データセットの行数を考える
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [24]:
def create_dataloader_v1(txt, 
                         batch_size=4,# 一つのテンソルのサイズ
                         max_length=256,
                         stride=128, # 入力(出力)token内の次の インデックスまでのジャンプ
                         shuffle=True,
                         drop_last=True, 
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset, 
        batch_size = batch_size, 
        shuffle=shuffle,
        drop_last = drop_last,
        num_workers=num_workers
    )
    
    return dataloader

In [25]:
with open("src/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 1,
    max_length = 4,
    stride = 1,
    shuffle = False    
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print('入力 token の tensor, 出力 token の tensor')
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

入力 token の tensor, 出力 token の tensor
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [26]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 1,
    max_length = 4,
    stride = 1,
    shuffle = False    
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [28]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 5,
    max_length = 4,
    stride = 1,
    shuffle = False    
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402],
        [1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899]])]
[tensor([[ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026],
        [ 2138,   257,  7026, 15632]]), tensor([[  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026],
        [ 2138,   257,  7026, 15632],
        [  257,  7026, 15632,   438]])]


- stride が小さいとオーバーラップ（モデルが同じデータを何度も見る）が起きてしまい，過剰適合
- バッチサイズは小さい方がメモリが小さくて済むが、勾配の計算やパラメータ計算が不安定になりノイズが増える．
- トレードオフなので実験すべきハイパーパラメータである．

In [29]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Inputs: \n{inputs}")
print(f"\nTagets: \n{targets}")

Inputs: 
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Tagets: 
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# 2.7 トークン埋め込みを作成する

token ID (`inputs_ids`) を埋め込み層 (`embedding_layer`) に基づいてベクトル空間に埋め込む。  

In [41]:
input_ids = torch.tensor([2, 3, 5, 1])

In [37]:
vocab_size = 6
output_dim = 3

埋め込み層 (`embedding_layer`) はランダマイズしておく．

In [44]:
torch.manual_seed(123) # rondom seed の固定
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) # サイズ固定するだけで勝手にランダマイズされる．
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [46]:
# inputids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


埋め込み層では、トークンID に対してその位置に依存しないようにベクトルを対応させる．

- 相対位置埋め込み: トークン間の相対的な位置（距離）に関連付ける
- 絶対位置埋め込み: シーケンス内の特定の位置に直接関連付ける * (こちらを採用)

In [51]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

各バッチごとに Token ID を出力

In [72]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, 
    shuffle = False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter) # verdict から得られる最初のバッチを取り出している

print("Token IDs:\n", inputs)
print("\nInputs shape\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape
 torch.Size([8, 4])


In [73]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [74]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length)) # プレースホルダベクトル [0, context_length-1]^d
print(pos_embeddings.shape)

torch.Size([4, 256])


トークン埋め込みに位置埋め込みを加算するの図

In [77]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [78]:
# input_embeddings[0]
